In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skmultilearn.model_selection import iterative_train_test_split
from wandb.keras import WandbCallback
import wandb

2022-06-19 19:20:06.267887: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
tf.__version__

'2.4.1'

In [4]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'pretrain_net': {
            'values': ['inception', 'resnet']
        },
        'batch_size': {
            'values': [20, 50]
        },
        'dropout': {
            'values': [0.2, 0.5]
        },
        'dense': {
            'values': [32, 256, 1024]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-5]
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'rmsprop']
        },
        'activation': {
            'values': ['relu', 'elu', 'selu', 'softmax']
        }
    }
}

In [5]:
IMAGE_SIZE = 320

In [6]:
df = pd.read_csv('/home/lab38/Multi_proj_6/data/straw_smallsizeimg_with_pest.csv')
df.head()

,image,grow,disease,area,points,original,disease-grow
0,/home/lab38/딸기/Strawberry Pest Damage_3.jpg,5,11,NaN,NaN,NaN,11-5
1,/home/lab38/딸기/Strawberry Pest Damage_727.jpg,2,11,NaN,NaN,NaN,11-2
2,/home/lab38/딸기/Strawberry Pest Damage_487.jpg,5,11,NaN,NaN,NaN,11-5
3,/home/lab38/딸기/Strawberry Pest Damage_124.jpg,5,11,NaN,NaN,NaN,11-5
4,/home/lab38/딸기/Strawberry Pest Damage_70.jpg,5,11,NaN,NaN,NaN,11-5


In [7]:
# label encoding
disease_encoder = LabelEncoder()
disease_encoder.fit(df['disease'])
df['disease'] = disease_encoder.transform(df['disease'])
print(df['disease'].unique())

grow_encoder = LabelEncoder()
grow_encoder.fit(df['grow'])
df['grow'] = grow_encoder.transform(df['grow'])
print(df['grow'].unique())

[3 0 1 2]
[4 1 3 0 2]


In [8]:
shuffled_df = df.sample(frac=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['image'],
                                                   df['disease-grow'],
                                                   stratify=df['disease-grow'],
                                                   test_size=0.2)

In [10]:
train_df = df[df['image'].isin(X_train)]
test_df = df[df['image'].isin(X_test)]

In [11]:
print(train_df['disease'].unique())
print(train_df['grow'].unique())
print(test_df['disease'].unique())
print(test_df['grow'].unique())

[3 0 1 2]
[4 1 3 2 0]
[3 0 1 2]
[3 0 1 4 2]


In [12]:
train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=20, # 최대 20도까지 회전
                            width_shift_range=0.1, # 최대 x 범위안에서 좌우/상하 이동
                            height_shift_range=0.1,
                            zoom_range=0.2, # 확대 축소 비율,
                            horizontal_flip=True, # 좌우반전
                            vertical_flip=True, # 상하반전
                            fill_mode='nearest')
valid_gen = ImageDataGenerator(rescale= 1. /255.)

In [13]:
def train() :
    wandb.init()
    tf.keras.backend.clear_session()
    config_defaults = {
        'pretrain_net': 'inception',
        'epochs' : 10,
        'batch_size': 50,
        'dropout' : 0.5,
        'learning_rate' : 1e-3,
        'activation': 'relu',
        'optimizer': 'adam'
    }
    
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_generator = train_gen.flow_from_dataframe(train_df, 
                                               x_col='image',
                                               y_col=['disease', 'grow'],
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='multi_output',
                                               batch_size=config.batch_size)
    valid_generator = valid_gen.flow_from_dataframe(test_df,
                                               x_col='image',
                                               y_col=['disease','grow'],
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='multi_output',
                                               batch_size=config.batch_size)
    if config.pretrain_net == 'inception' :
        MODEL_IMAGE_SIZE = 299
        base_model = inception_resnet_v2.InceptionResNetV2(
            weights='imagenet',
            include_top = False,
            input_shape = (MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE,3)
        )
    elif config.pretrain_net == 'resnet' :
        MODEL_IMAGE_SIZE = 224
        base_model= resnet50.ResNet50(
              weights='imagenet',
              include_top=False,
              input_shape = (MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE,3)
             )
    base_model.trainable = False 

    
    # resizing model
    input_data = layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3))
    x = tf.keras.layers.experimental.preprocessing.Resizing(MODEL_IMAGE_SIZE, MODEL_IMAGE_SIZE)(input_data)
    resizing = Model(inputs=input_data, outputs=x, name='resize')
   
    # model
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = resizing(inputs)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(config.dropout)(x)
    backbone_out = layers.Dense(config.dense, activation=config.activation)(x)

    disease_outputs = layers.Dense(df['disease'].nunique(), activation='softmax',
                            name = 'diease_outputs')(backbone_out)
    grow_outputs = layers.Dense(df['grow'].nunique(), activation='softmax',
                        name = 'grow_outputs')(backbone_out)

    model = Model(inputs=inputs, 
                  outputs=[disease_outputs, grow_outputs],
                  name='strawberry')    
    if config.optimizer=='sgd':
        optimizer = SGD(learning_rate=config.learning_rate)
    elif config.optimizer=='rmsprop':
        optimizer = RMSprop(learning_rate=config.learning_rate)
    elif config.optimizer=='adam':
        optimizer = Adam(learning_rate=config.learning_rate)

    model.compile(loss={
                  'diease_outputs' : 'sparse_categorical_crossentropy',
                  'grow_outputs' : 'sparse_categorical_crossentropy'
                  },
                  optimizer=optimizer,
                  metrics=['accuracy'])    
    
    _ = model.fit(train_generator,
          validation_data=valid_generator,
          verbose=1,
          epochs=config.epochs,
          callbacks=[WandbCallback()],
          steps_per_epoch=len(train_df)//config.batch_size)


In [14]:
sweep_id = wandb.sweep(sweep_config, project='strawberry')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: p0s861p4
Sweep URL: https://wandb.ai/hkleee/strawberry/sweeps/p0s861p4


In [ ]:
wandb.agent('natutesf', train)

wandb: Agent Starting Run: kra07bbi with config:
wandb: 	activation: elu
wandb: 	batch_size: 50
wandb: 	dense: 32
wandb: 	dropout: 0.2
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: sgd
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hkleee. Use `wandb login --relogin` to force relogin


Found 13437 validated image filenames.
Found 3360 validated image filenames.


2022-06-19 19:20:26.396456: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-19 19:20:26.397593: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-19 19:20:26.440791: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-19 19:20:26.441449: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-06-19 19:20:26.441503: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-19 19:20:26.517781: I tensorflow/stream_executor/platform/default/dso_loade

Epoch 1/10


2022-06-19 19:20:43.842360: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-06-19 19:20:44.897975: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


...

wandb: Agent Starting Run: sxrohti5 with config:
wandb: 	activation: softmax
wandb: 	batch_size: 50
wandb: 	dense: 32
wandb: 	dropout: 0.2
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: sgd
wandb: 	pretrain_net: resnet
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
268/268 [==============================] - 301s 1s/step - loss: 3.0234 - diease_outputs_loss: 1.3962 - grow_outputs_loss: 1.6272 - diease_outputs_accuracy: 0.2641 - grow_outputs_accuracy: 0.1278 - val_loss: 3.0198 - val_diease_outputs_loss: 1.3936 - val_grow_outputs_loss: 1.6262 - val_diease_outputs_accuracy: 0.2658 - val_grow_outputs_accuracy: 0.1443
Epoch 2/10
268/268 [==============================] - 298s 1s/step - loss: 3.0203 - diease_outputs_loss: 1.3942 - grow_outputs_loss: 1.6261 - diease_outputs_accuracy: 0.2735 - grow_outputs_accuracy: 0.1299 - val_loss: 3.0166 - val_diease_outputs_loss: 1.3911 - val_grow_outputs_loss: 1.6255 - val_diease_outputs_accuracy: 0.2658 - val_grow_outputs_accuracy: 0.1298
Epoch 3/10
268/268 [==============================] - 299s 1s/step - loss: 3.0181 - diease_outputs_loss: 1.3916 - grow_outputs_loss: 1.6265 - diease_outputs_accuracy: 0.2904 - grow_outputs_accu

diease_outputs_accuracy,▁▂▃▃▄▅▆▆▇█
diease_outputs_loss,█▇▇▆▅▄▄▃▂▁
epoch,▁▂▃▃▄▅▆▆▇█
grow_outputs_accuracy,██▅▃▇▅▅▅▁▂
grow_outputs_loss,█▇▇▆▅▄▃▂▁▁
loss,█▇▇▆▅▄▃▃▂▁
val_diease_outputs_accuracy,▁▁▁▂▄▆▇███
val_diease_outputs_loss,█▇▇▆▅▄▄▃▂▁
val_grow_outputs_accuracy,█▇▅▄▂▁▁▁▁▁
val_grow_outputs_loss,█▇▆▆▅▄▃▃▂▁
val_loss,█▇▇▆▅▄▃▃▂▁


wandb: Agent Starting Run: m8oxkhz3 with config:
wandb: 	activation: elu
wandb: 	batch_size: 20
wandb: 	dense: 1024
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd
wandb: 	pretrain_net: inception
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Found 13437 validated image filenames.
Found 3360 validated image filenames.
Epoch 1/10
671/671 [==============================] - 326s 473ms/step - loss: 2.2837 - diease_outputs_loss: 0.9925 - grow_outputs_loss: 1.2912 - diease_outputs_accuracy: 0.5718 - grow_outputs_accuracy: 0.4477 - val_loss: 1.6294 - val_diease_outputs_loss: 0.6822 - val_grow_outputs_loss: 0.9473 - val_diease_outputs_accuracy: 0.7589 - val_grow_outputs_accuracy: 0.6223
Epoch 2/10
671/671 [==============================] - 313s 467ms/step - loss: 1.6591 - diease_outputs_loss: 0.7103 - grow_outputs_loss: 0.9487 - diease_outputs_accuracy: 0.7289 - grow_outputs_accuracy: 0.6090 - val_loss: 1.4771 - val_diease_outputs_loss: 0.5976 - val_grow_outputs_loss: 0.8796 - val_diease_outputs_accuracy: 0.7887 - val_grow_outputs_accuracy: 0.6271
Epoch 3/10
671/671 [==============================] - 317s 472ms/step - loss: 1.5188 - diease_outputs_loss: 0.6304 - grow_outputs_loss: 0.8884 - diease_outputs_accuracy: 0.7653 - grow_out

In [ ]:
# MLOPs